In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

746

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=e632494dd98e7fac91fec90318470826


In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    time.sleep(1)
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"] 
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_wdescription =city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_wdescription})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | avarua
Processing Record 2 of Set 1 | jamestown
Processing Record 3 of Set 1 | ochakiv
Processing Record 4 of Set 1 | samusu
City not found. Skipping...
Processing Record 5 of Set 1 | chokurdakh
Processing Record 6 of Set 1 | kodinsk
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | bengkulu
Processing Record 9 of Set 1 | haines junction
Processing Record 10 of Set 1 | bethel
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | saint-philippe
Processing Record 13 of Set 1 | castro
Processing Record 14 of Set 1 | nabire
Processing Record 15 of Set 1 | tuktoyaktuk
Processing Record 16 of Set 1 | souillac
Processing Record 17 of Set 1 | albany
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | kieta
Processing Record 20 of Set 1 | ostrovnoy
Processing Record 21 of Set 1 | vila franca do campo
Processing Record 22 of Set 1 | norman well

Processing Record 36 of Set 4 | eureka
Processing Record 37 of Set 4 | chaa-khol
City not found. Skipping...
Processing Record 38 of Set 4 | doha
Processing Record 39 of Set 4 | lasa
Processing Record 40 of Set 4 | santa ana
Processing Record 41 of Set 4 | boa vista
Processing Record 42 of Set 4 | formoso do araguaia
City not found. Skipping...
Processing Record 43 of Set 4 | zhangye
Processing Record 44 of Set 4 | miri
Processing Record 45 of Set 4 | kamenskoye
City not found. Skipping...
Processing Record 46 of Set 4 | nanortalik
Processing Record 47 of Set 4 | galle
Processing Record 48 of Set 4 | victoria
Processing Record 49 of Set 4 | mutsamudu
City not found. Skipping...
Processing Record 50 of Set 4 | belushya guba
City not found. Skipping...
Processing Record 1 of Set 5 | pizarro
Processing Record 2 of Set 5 | yibin
Processing Record 3 of Set 5 | jilib
Processing Record 4 of Set 5 | san quintin
Processing Record 5 of Set 5 | nikolskoye
Processing Record 6 of Set 5 | flinders
P

Processing Record 20 of Set 8 | zigong
Processing Record 21 of Set 8 | whitehorse
Processing Record 22 of Set 8 | karakendzha
City not found. Skipping...
Processing Record 23 of Set 8 | rupert
Processing Record 24 of Set 8 | hamilton
Processing Record 25 of Set 8 | praia da vitoria
Processing Record 26 of Set 8 | berlevag
Processing Record 27 of Set 8 | samarinda
Processing Record 28 of Set 8 | san policarpo
Processing Record 29 of Set 8 | paragould
Processing Record 30 of Set 8 | falealupo
City not found. Skipping...
Processing Record 31 of Set 8 | olavarria
Processing Record 32 of Set 8 | kavaratti
Processing Record 33 of Set 8 | hualmay
Processing Record 34 of Set 8 | nyurba
Processing Record 35 of Set 8 | gimli
Processing Record 36 of Set 8 | teshie
Processing Record 37 of Set 8 | nhulunbuy
Processing Record 38 of Set 8 | daru
Processing Record 39 of Set 8 | chalchihuites
Processing Record 40 of Set 8 | tapaua
City not found. Skipping...
Processing Record 41 of Set 8 | giraltovce
P

Processing Record 4 of Set 12 | saint-paul
Processing Record 5 of Set 12 | cedar city
Processing Record 6 of Set 12 | marang
Processing Record 7 of Set 12 | kikwit
Processing Record 8 of Set 12 | nakonde
Processing Record 9 of Set 12 | falkenberg
Processing Record 10 of Set 12 | leh
Processing Record 11 of Set 12 | thinadhoo
Processing Record 12 of Set 12 | keshabpur
Processing Record 13 of Set 12 | mutsu
Processing Record 14 of Set 12 | tumen
Processing Record 15 of Set 12 | bathsheba
Processing Record 16 of Set 12 | samalaeulu
City not found. Skipping...
Processing Record 17 of Set 12 | port-gentil
Processing Record 18 of Set 12 | kovdor
Processing Record 19 of Set 12 | anchorage
Processing Record 20 of Set 12 | namatanai
Processing Record 21 of Set 12 | chengdu
Processing Record 22 of Set 12 | cururupu
Processing Record 23 of Set 12 | yurya
City not found. Skipping...
Processing Record 24 of Set 12 | athabasca
Processing Record 25 of Set 12 | saint andrews
Processing Record 26 of Se

Processing Record 38 of Set 15 | tiarei
Processing Record 39 of Set 15 | agua blanca sur
Processing Record 40 of Set 15 | aden
Processing Record 41 of Set 15 | kartaly
Processing Record 42 of Set 15 | liaozhong
Processing Record 43 of Set 15 | stornoway
Processing Record 44 of Set 15 | santa isabel do rio negro
Processing Record 45 of Set 15 | amritsar
Processing Record 46 of Set 15 | bataipora
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Avarua,CK,-21.2078,-159.7750,77.05,73,40,9.22,scattered clouds
1,Jamestown,US,42.0970,-79.2353,85.32,44,1,8.05,clear sky
2,Ochakiv,UA,46.6128,31.5498,60.94,64,48,7.43,scattered clouds
3,Chokurdakh,RU,70.6333,147.9167,59.70,60,48,3.49,scattered clouds
4,Kodinsk,RU,58.6881,99.1844,43.16,60,100,3.22,overcast clouds
5,Punta Arenas,CL,-53.1500,-70.9167,37.51,87,40,6.71,scattered clouds
6,Bengkulu,ID,-3.8004,102.2655,74.98,92,18,3.53,few clouds
7,Haines Junction,CA,60.7522,-137.5108,60.75,39,79,11.50,light rain
8,Bethel,US,41.3712,-73.4140,91.54,46,1,10.36,clear sky
9,Rikitea,PF,-23.1203,-134.9692,75.81,80,100,10.92,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")